In [2]:
import os, sys
sys.path.insert(0, '/home/trax/trax/website')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "trax.settings")
import django
django.setup()

In [3]:
from fuzzywuzzy import fuzz, process
from tracks.models import Track
from vehicles.models import Vehicle, VehicleClass
from trax.utils import get_object_by_string

def lt2millis(parts):
    m, rest = parts.split(':')
    millis = 1000 * (int(m)*60 + float(rest))
    return int(millis)

def get_object_by_string(s, klass, attribute, score_cutoff=90):
    choices = klass.objects.all().values_list(attribute, flat=True)
    result = process.extractOne(s, choices,
                                score_cutoff=score_cutoff)
    if result:
        return klass.objects.get(**{attribute:result[0]})


In [10]:
def print_similar(name):
    for x in list(Vehicle.objects.filter(name__startswith=name).order_by('pk')):
        print("similar:", x.pk, x.cc_laptime_millis and x.cc_laptime or '?', x.cc_millis_per_km , x.name, x.laptime_set.all().count())


In [7]:
v = VehicleClass.objects.filter(name__icontains="sport")
print( [x.name for x in v] )


['Sport', 'Sports_Classic']


In [36]:
v = Vehicle.objects.filter(name__startswith="RE")
print( [(x.name, x.cc_laptime) for x in v])  # v.name

[('RE-7B (before Gunrunning)', '01:00.627'), ('RE-7B (do not use)', '01:02.096'), ('RE-7B [STOCK]', '01:04.431'), ('RE7B', '00:59.727')]


In [39]:
x=v[1]
x.name
x.classes.all()

<QuerySet [<VehicleClass: Super>]>

In [8]:
data = '''Sport	Ruston	1:08.103
Sport	Omnis	1:07.168
Sport	Comet Retro Custom	1:05.366
Sport	Specter Custom	1:04.332'''

In [15]:
for line in data.splitlines():
    vdata = line.split('\t')
    if vdata[2]:
        vv = get_object_by_string(vdata[1], Vehicle, "name", score_cutoff=99)
        if not vv:
            print(".............................NOT FOUND:", vdata[1] , "CREATING NOW.")
            vc = VehicleClass.objects.get(name=vdata[0])
            vv = Vehicle(name=vdata[1])
            vv.save()
            vv.classes.add(vc)
            vv.save()
        #else:
        #   raise NotImplementedError
        print ("vehicle:", vv.name, list(vv.classes.all()))
        millis = lt2millis(vdata[2])
        vv.cc_laptime_millis = millis
        try:
            vv.topspeed_kmh = float(vdata[3]) * 1.60934
        except:
            print("no topspeed")
            pass
        if (vdata[1] != vv.name):
            print("..... NOT MODIFIED:", vdata[1], vv.name, vv.pk, vv.cc_laptime_millis, vv.topspeed_kmh)
        else:
            vv.save()
            # print_similar(vdata[1])

vehicle: Ruston [<VehicleClass: Sport>]
no topspeed
vehicle: Omnis [<VehicleClass: Sport>]
no topspeed
vehicle: Comet Retro Custom [<VehicleClass: Sport>]
no topspeed
vehicle: Specter Custom [<VehicleClass: Sport>]
no topspeed


In [26]:
v = Vehicle.objects.get(name="RE-7B")
v.name = "RE-7B (do not use)"
v.save()

In [24]:
g = Vehicle.objects.get(name="811")
g.cc_laptime, g.cc_millis_per_km, g.cc_laptime_millis, g.topspeed_kmh, g.topspeed_kmh/1.60934

('01:03.200', 24401, 63200, 208.007195, 129.25)

In [71]:
Vehicle.objects.exclude(classes__in=vc)

<QuerySet []>

In [8]:
v = Vehicle.objects.get(name="Ruston")
v.classes.add(sp)
v.save()

In [7]:
from vehicles.models import VehicleClass
vc  = list(VehicleClass.objects.all().values_list("pk", flat=True))
muscle = VehicleClass.objects.get(name="Muscle")
off = VehicleClass.objects.get(name="Offroad")
suv = VehicleClass.objects.get(name="SUV")
mo = VehicleClass.objects.get(name="Motorcycle")
sp = VehicleClass.objects.get(name="Sport")

In [44]:
vc

[9, 11, 12, 8, 3, 10, 5, 4, 7, 2, 1, 17, 6]

In [9]:
a, = Vehicle.objects.filter(name="Ruston")

In [10]:
a.classes.all()

<QuerySet [<VehicleClass: Sport>]>

In [24]:
tc = get_object_by_string( "Turismo Classic", Vehicle,"name")
tc.classes.all()

<QuerySet [<VehicleClass: Sports_Classic>]>

DoesNotExist: Vehicle matching query does not exist.

In [8]:
inf = v[0]

In [4]:
vc = VehicleClass.objects.get(name__startswith="Spor", name__icontains="lassi")

In [5]:
vc

<VehicleClass: Sports_Classic>

In [7]:
tor = Vehicle(name="Torero")

In [9]:
tor.topspeed_kmh=187.5
tor.save()
tor.classes.add(vc)

In [10]:
tor.save()

In [13]:
v = Vehicle.objects.get(name="Omnis")
v.classes.all()

<QuerySet [<VehicleClass: Sport>]>